In [1]:
from __future__ import division
import os
import cv2
import numpy as np
import sys
import pickle
from optparse import OptionParser
import time
from keras_frcnn import config
from keras import backend as K
from keras.layers import Input
from keras.models import Model
from keras_frcnn import roi_helpers


Using TensorFlow backend.


In [2]:
sys.setrecursionlimit(40000)

In [3]:
def format_img_size(img, C):
    """ formats the image size based on config """
    img_min_side = float(C.im_size)
    (height,width,_) = img.shape

    if width <= height:
        ratio = img_min_side/width
        new_height = int(ratio * height)
        new_width = int(img_min_side)
    else:
        ratio = img_min_side/height
        new_width = int(ratio * width)
        new_height = int(img_min_side)
    img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_CUBIC)
    return img, ratio

In [4]:
def format_img_channels(img, C):
    """ formats the image channels based on config """
    img = img[:, :, (2, 1, 0)]
    img = img.astype(np.float32)
    img[:, :, 0] -= C.img_channel_mean[0]
    img[:, :, 1] -= C.img_channel_mean[1]
    img[:, :, 2] -= C.img_channel_mean[2]
    img /= C.img_scaling_factor
    img = np.transpose(img, (2, 0, 1))
    img = np.expand_dims(img, axis=0)
    return img


In [5]:
def format_img(img, C):
    """ formats an image for model prediction based on config """
    img, ratio = format_img_size(img, C)
    img = format_img_channels(img, C)
    return img, ratio

In [6]:
def get_real_coordinates(ratio, x1, y1, x2, y2):

    real_x1 = int(round(x1 // ratio))
    real_y1 = int(round(y1 // ratio))
    real_x2 = int(round(x2 // ratio))
    real_y2 = int(round(y2 // ratio))

    return (real_x1, real_y1, real_x2 ,real_y2)

In [7]:
test_path = '/home/wenjie/Desktop/earImageDataset/re_image/'
num_rois = 32
config_filename = "config.pickle"
network = 'vgg'

config_output_filename = config_filename
with open(config_output_filename, 'rb') as f_in:
    C = pickle.load(f_in)
if C.network == 'resnet50':
    import keras_frcnn.resnet as nn
elif C.network == 'vgg':
    import keras_frcnn.vgg as nn
C.use_horizontal_flips = False
C.use_vertical_flips = False
C.rot_90 = False

img_path = test_path

class_mapping = C.class_mapping

if 'bg' not in class_mapping:
	class_mapping['bg'] = len(class_mapping)

class_mapping = {v: k for k, v in class_mapping.items()}
print(class_mapping)
class_to_color = {class_mapping[v]: np.random.randint(0, 255, 3) for v in class_mapping}
C.num_rois = int(num_rois)

if C.network == 'resnet50':
    num_features = 1024
elif C.network == 'vgg':
    num_features = 512

if K.image_dim_ordering() == 'th':
    input_shape_img = (3, None, None)
    input_shape_features = (num_features, None, None)
else:
    input_shape_img = (None, None, 3)
    input_shape_features = (None, None, num_features)


img_input = Input(shape=input_shape_img)
roi_input = Input(shape=(C.num_rois, 4))
feature_map_input = Input(shape=input_shape_features)

# define the base network (resnet here, can be VGG, Inception, etc)
shared_layers = nn.nn_base(img_input, trainable=True)

# define the RPN, built on the base layers
num_anchors = len(C.anchor_box_scales) * len(C.anchor_box_ratios)
rpn_layers = nn.rpn(shared_layers, num_anchors)

classifier = nn.classifier(feature_map_input, roi_input, C.num_rois, nb_classes=len(class_mapping), trainable=True)

model_rpn = Model(img_input, rpn_layers)
model_classifier_only = Model([feature_map_input, roi_input], classifier)

model_classifier = Model([feature_map_input, roi_input], classifier)

print('Loading weights from {}'.format(C.model_path))
model_rpn.load_weights(C.model_path, by_name=True)
model_classifier.load_weights(C.model_path, by_name=True)

model_rpn.compile(optimizer='sgd', loss='mse')
model_classifier.compile(optimizer='sgd', loss='mse')

{0: 'Face', 1: 'bg'}
Loading weights from ./model_frcnn.hdf5


In [13]:


all_imgs = []

classes = {}

bbox_threshold = 0.8

visualise = True

for idx, img_name in enumerate(sorted(os.listdir(img_path))):
    if not img_name.lower().endswith(('.bmp', '.jpeg', '.jpg', '.png', '.tif', '.tiff')):
        continue
    print(img_name)
    st = time.time()
    filepath = os.path.join(img_path,img_name)

    img = cv2.imread(filepath)

    X, ratio = format_img(img, C)

    if K.image_dim_ordering() == 'tf':
        X = np.transpose(X, (0, 2, 3, 1))

    # get the feature maps and output from the RPN
    [Y1, Y2, F] = model_rpn.predict(X)


    R = roi_helpers.rpn_to_roi(Y1, Y2, C, K.image_dim_ordering(), overlap_thresh=0.7)

    # convert from (x1,y1,x2,y2) to (x,y,w,h)
    R[:, 2] -= R[:, 0]
    R[:, 3] -= R[:, 1]

    # apply the spatial pyramid pooling to the proposed regions
    bboxes = {}
    probs = {}

    for jk in range(R.shape[0]//C.num_rois + 1):
        ROIs = np.expand_dims(R[C.num_rois*jk:C.num_rois*(jk+1), :], axis=0)
        if ROIs.shape[1] == 0:
            break

        if jk == R.shape[0]//C.num_rois:
            #pad R
            curr_shape = ROIs.shape
            target_shape = (curr_shape[0],C.num_rois,curr_shape[2])
            ROIs_padded = np.zeros(target_shape).astype(ROIs.dtype)
            ROIs_padded[:, :curr_shape[1], :] = ROIs
            ROIs_padded[0, curr_shape[1]:, :] = ROIs[0, 0, :]
            ROIs = ROIs_padded

        [P_cls, P_regr] = model_classifier_only.predict([F, ROIs])

        for ii in range(P_cls.shape[1]):

            if np.max(P_cls[0, ii, :]) < bbox_threshold or np.argmax(P_cls[0, ii, :]) == (P_cls.shape[2] - 1):
                continue

            cls_name = class_mapping[np.argmax(P_cls[0, ii, :])]

            if cls_name not in bboxes:
                bboxes[cls_name] = []
                probs[cls_name] = []

            (x, y, w, h) = ROIs[0, ii, :]

            cls_num = np.argmax(P_cls[0, ii, :])
            try:
                (tx, ty, tw, th) = P_regr[0, ii, 4*cls_num:4*(cls_num+1)]
                tx /= C.classifier_regr_std[0]
                ty /= C.classifier_regr_std[1]
                tw /= C.classifier_regr_std[2]
                th /= C.classifier_regr_std[3]
                x, y, w, h = roi_helpers.apply_regr(x, y, w, h, tx, ty, tw, th)
            except:
                pass
            bboxes[cls_name].append([C.rpn_stride*x, C.rpn_stride*y, C.rpn_stride*(x+w), C.rpn_stride*(y+h)])
            probs[cls_name].append(np.max(P_cls[0, ii, :]))

    all_dets = []

    for key in bboxes:
        bbox = np.array(bboxes[key])

        new_boxes, new_probs = roi_helpers.non_max_suppression_fast(bbox, np.array(probs[key]), overlap_thresh=0.5)

        
        for jk in range(new_boxes.shape[0]):
            (x1, y1, x2, y2) = new_boxes[jk,:]
            if new_probs[jk] < 0.95:
                continue
            (real_x1, real_y1, real_x2, real_y2) = get_real_coordinates(ratio, x1, y1, x2, y2)

            #cv2.rectangle(img,(real_x1, real_y1), (real_x2, real_y2), (int(class_to_color[key][0]), int(class_to_color[key][1]), int(class_to_color[key][2])),2)

            textLabel = '{}: {}'.format(key,int(100*new_probs[jk]))
            all_dets.append((key,100*new_probs[jk]))

            (retval,baseLine) = cv2.getTextSize(textLabel,cv2.FONT_HERSHEY_COMPLEX,1,1)
            textOrg = (real_x1, real_y1-0)

            #cv2.rectangle(img, (textOrg[0] - 5, textOrg[1]+baseLine - 5), (textOrg[0]+retval[0] + 5, textOrg[1]-retval[1] - 5), (0, 0, 0), 2)
            #cv2.rectangle(img, (textOrg[0] - 5, textOrg[1]+baseLine - 5), (textOrg[0]+retval[0] + 5, textOrg[1]-retval[1] - 5), (255, 255, 255), -1)
            #cv2.putText(img, textLabel, textOrg, cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 0), 1)

    croped_img = img[real_y1:real_y2,real_x1:real_x2]

    print('Elapsed time = {}'.format(time.time() - st))
    print(all_dets)
    cv2.imwrite('../cropped_ears/'+img_name,croped_img)
    #cv2.imshow('img', img)
    #cv2.waitKey(0)

001_.jpg
Elapsed time = 0.24910664558410645
[('Face', 99.98691082000732)]
001_d.jpg
Elapsed time = 0.23235511779785156
[('Face', 99.99669790267944)]
001_dt.jpg
Elapsed time = 0.22949767112731934
[('Face', 99.99203681945801)]
001_t.jpg
Elapsed time = 0.2434535026550293
[('Face', 99.88282322883606)]
002_.jpg
Elapsed time = 0.23749828338623047
[('Face', 99.98804330825806)]
002_d.jpg
Elapsed time = 0.25582146644592285
[('Face', 99.99321699142456)]
002_dt.jpg
Elapsed time = 0.22513747215270996
[('Face', 99.99313354492188)]
002_t.jpg
Elapsed time = 0.22369980812072754
[('Face', 99.97166991233826)]
003_.jpg
Elapsed time = 0.24251461029052734
[('Face', 99.97288584709167)]
003_d.jpg
Elapsed time = 0.22764039039611816
[('Face', 99.99712705612183)]
003_dt.jpg
Elapsed time = 0.21316051483154297
[('Face', 99.98883008956909)]
003_t.jpg
Elapsed time = 0.22950148582458496
[('Face', 99.92280006408691)]
004_.jpg
Elapsed time = 0.2335507869720459
[('Face', 99.9915361404419)]
004_d.jpg
Elapsed time = 0.21

Elapsed time = 0.2229619026184082
[('Face', 99.9894380569458)]
028_t.jpg
Elapsed time = 0.23577070236206055
[('Face', 99.86293315887451)]
029_.jpg
Elapsed time = 0.22696542739868164
[('Face', 99.84468817710876)]
029_d.jpg
Elapsed time = 0.22890424728393555
[('Face', 99.98999834060669)]
029_dt.jpg
Elapsed time = 0.22893309593200684
[('Face', 99.9776303768158)]
029_t.jpg
Elapsed time = 0.22964072227478027
[('Face', 99.89702701568604)]
030_.jpg
Elapsed time = 0.22323822975158691
[('Face', 99.96616840362549)]
030_d.jpg
Elapsed time = 0.23242950439453125
[('Face', 99.95492100715637)]
030_dt.jpg
Elapsed time = 0.22747254371643066
[('Face', 99.88634586334229)]
030_t.jpg
Elapsed time = 0.23479199409484863
[]
031_.jpg
Elapsed time = 0.22051215171813965
[('Face', 99.99160766601562)]
031_d.jpg
Elapsed time = 0.2332901954650879
[('Face', 99.99265670776367)]
031_dt.jpg
Elapsed time = 0.21734833717346191
[('Face', 99.99305009841919)]
031_t.jpg
Elapsed time = 0.2398545742034912
[('Face', 99.971657991

Elapsed time = 0.2229292392730713
[('Face', 99.95675683021545)]
056_dt.jpg
Elapsed time = 0.23794889450073242
[('Face', 99.9434769153595)]
056_t.jpg
Elapsed time = 0.23216533660888672
[('Face', 99.8609185218811)]
057_.jpg
Elapsed time = 0.22340607643127441
[('Face', 99.91344809532166)]
057_d.jpg
Elapsed time = 0.22702479362487793
[('Face', 99.97336268424988)]
057_dt.jpg
Elapsed time = 0.23210501670837402
[('Face', 99.98499155044556)]
057_t.jpg
Elapsed time = 0.2281782627105713
[('Face', 99.05732274055481)]
058_.jpg
Elapsed time = 0.2254931926727295
[('Face', 99.78001713752747)]
058_d.jpg
Elapsed time = 0.23212552070617676
[('Face', 99.99785423278809)]
058_dt.jpg
Elapsed time = 0.23164033889770508
[('Face', 99.99381303787231)]
058_t.jpg
Elapsed time = 0.22732782363891602
[('Face', 99.99309778213501)]
059_.jpg
Elapsed time = 0.22028088569641113
[('Face', 99.97525811195374)]
059_d.jpg
Elapsed time = 0.22699904441833496
[('Face', 99.94984865188599)]
059_dt.jpg
Elapsed time = 0.227834463119

Elapsed time = 0.22477936744689941
[('Face', 99.9798595905304)]
084_.jpg
Elapsed time = 0.24031376838684082
[('Face', 99.98635053634644)]
084_d.jpg
Elapsed time = 0.22823143005371094
[('Face', 99.99421834945679)]
084_dt.jpg
Elapsed time = 0.24373888969421387
[('Face', 99.99901056289673)]
084_t.jpg
Elapsed time = 0.2248084545135498
[('Face', 99.98550415039062)]
085_.jpg
Elapsed time = 0.23537826538085938
[('Face', 99.95390176773071)]
085_d.jpg
Elapsed time = 0.22773432731628418
[('Face', 99.99337196350098)]
085_dt.jpg
Elapsed time = 0.23179984092712402
[('Face', 99.99455213546753)]
085_t.jpg
Elapsed time = 0.23077082633972168
[('Face', 99.9901533126831)]
086_.jpg
Elapsed time = 0.2333512306213379
[('Face', 99.93120431900024)]
086_d.jpg
Elapsed time = 0.2284224033355713
[('Face', 99.97214674949646)]
086_dt.jpg
Elapsed time = 0.22277593612670898
[('Face', 99.98354911804199)]
086_t.jpg
Elapsed time = 0.22330999374389648
[('Face', 99.96836185455322)]
087_.jpg
Elapsed time = 0.23362970352172

Elapsed time = 0.21902227401733398
[('Face', 99.99910593032837)]
111_dt.jpg
Elapsed time = 0.2401444911956787
[('Face', 99.99926090240479)]
111_t.jpg
Elapsed time = 0.23200678825378418
[('Face', 99.99310970306396)]
112_.jpg
Elapsed time = 0.23250341415405273
[('Face', 99.85931515693665)]
112_d.jpg
Elapsed time = 0.2249283790588379
[('Face', 99.98822212219238)]
112_dt.jpg
Elapsed time = 0.23182964324951172
[('Face', 99.9745786190033)]
112_t.jpg
Elapsed time = 0.22789764404296875
[('Face', 99.81512427330017)]
113_.jpg
Elapsed time = 0.23345303535461426
[('Face', 99.48201179504395)]
113_d.jpg
Elapsed time = 0.23170733451843262
[('Face', 99.9921441078186)]
113_dt.jpg
Elapsed time = 0.2279644012451172
[('Face', 99.98040795326233)]
113_t.jpg
Elapsed time = 0.2291858196258545
[('Face', 99.90480542182922)]
114_.jpg
Elapsed time = 0.23105573654174805
[('Face', 99.67369437217712)]
114_d.jpg
Elapsed time = 0.22705960273742676
[('Face', 99.99698400497437)]
114_dt.jpg
Elapsed time = 0.2243444919586

Elapsed time = 0.22765302658081055
[('Face', 99.7703492641449)]
139_d.jpg
Elapsed time = 0.22580552101135254
[('Face', 99.989914894104)]
139_dt.jpg
Elapsed time = 0.21958208084106445
[('Face', 99.99467134475708)]
139_t.jpg
Elapsed time = 0.22739553451538086
[('Face', 99.96941089630127)]
140_.jpg
Elapsed time = 0.22993040084838867
[('Face', 99.60110187530518)]
140_d.jpg
Elapsed time = 0.2308192253112793
[('Face', 99.98955726623535)]
140_dt.jpg
Elapsed time = 0.22921133041381836
[('Face', 99.99133348464966)]
140_t.jpg
Elapsed time = 0.22530674934387207
[('Face', 97.99740314483643)]
141_.jpg
Elapsed time = 0.23543071746826172
[('Face', 99.98880624771118)]
141_d.jpg
Elapsed time = 0.22626662254333496
[('Face', 99.9951958656311)]
141_dt.jpg
Elapsed time = 0.2277228832244873
[('Face', 99.98311996459961)]
141_t.jpg
Elapsed time = 0.2368466854095459
[('Face', 99.98419284820557)]
142_.jpg
Elapsed time = 0.22764921188354492
[('Face', 99.92353916168213)]
142_d.jpg
Elapsed time = 0.230341911315917

Elapsed time = 0.23369836807250977
[('Face', 99.99657869338989)]
166_t.jpg
Elapsed time = 0.23544526100158691
[('Face', 99.84671473503113)]
167_.jpg
Elapsed time = 0.23055100440979004
[('Face', 99.92536902427673)]
167_d.jpg
Elapsed time = 0.22834253311157227
[('Face', 99.98929500579834)]
167_dt.jpg
Elapsed time = 0.21804428100585938
[('Face', 99.97733235359192)]
167_t.jpg
Elapsed time = 0.22069025039672852
[('Face', 99.71169233322144)]
168_.jpg
Elapsed time = 0.2259387969970703
[('Face', 99.62410926818848)]
168_d.jpg
Elapsed time = 0.2279224395751953
[('Face', 99.98561143875122)]
168_dt.jpg
Elapsed time = 0.23427176475524902
[('Face', 99.99233484268188)]
168_t.jpg
Elapsed time = 0.23071575164794922
[('Face', 99.25479292869568)]
169_.jpg
Elapsed time = 0.23987245559692383
[('Face', 99.62579011917114)]
169_d.jpg
Elapsed time = 0.23145413398742676
[('Face', 99.97969269752502)]
169_dt.jpg
Elapsed time = 0.2291579246520996
[('Face', 99.98703002929688)]
169_t.jpg
Elapsed time = 0.23035550117

Elapsed time = 0.22044944763183594
[('Face', 99.99088048934937)]
194_d.jpg
Elapsed time = 0.2232351303100586
[('Face', 99.99860525131226)]
194_dt.jpg
Elapsed time = 0.2328639030456543
[('Face', 99.99504089355469)]
194_t.jpg
Elapsed time = 0.22964119911193848
[('Face', 99.94433522224426)]
195_.jpg
Elapsed time = 0.23399829864501953
[('Face', 99.93163347244263)]
195_d.jpg
Elapsed time = 0.22800755500793457
[('Face', 99.99855756759644)]
195_dt.jpg
Elapsed time = 0.22672414779663086
[('Face', 99.98996257781982)]
195_t.jpg
Elapsed time = 0.2202777862548828
[('Face', 99.94240999221802)]
